In [2]:
import os
import numpy as np
from PIL import Image

def save_images_labels(images: np.ndarray, labels:np.ndarray, output_dir: str, label_dir, offset = 0, explicit_split = None):
    assert images.ndim == 4 and images.shape[1] == 28 and images.shape[2] == 28 and images.shape[3] == 3, "Input must be (N, 28, 28, 3)"
    assert labels.ndim == 2 and labels.shape[1] == 4, "Labels must be (N, 4)" 

    n_total = images.shape[0]
    n_train = int(0.7 * n_total)
    n_val = int(0.2 * n_total)
    n_test = n_total - n_train - n_val

    # Define split points
    split_points = [0, n_train, n_train + n_val, n_total]
    split_names = ['train', 'validation', 'test']

    # Create output folders
    for split in split_names:
        split_path = os.path.join(output_dir, split)
        os.makedirs(split_path, exist_ok=True)

    # Save images
    for idx in range(n_total):
        if idx < split_points[1]:
            split = 'train'
        elif idx < split_points[2]:
            split = 'validation'
        else:
            split = 'test'
        if explicit_split != None: 
            split = explicit_split 
        img = Image.fromarray(images[idx].astype(np.uint8))
        img_path = os.path.join(output_dir, split, f'image_{offset+idx:06d}.png')
        img.save(img_path)
        #now do the same for label - space seperates it 
        label = labels[idx]
        label_path = os.path.join(label_dir, split, f'image_{offset+idx:06d}.txt')
        if np.all(label == 0):
            open(label_path, 'w').close()  # Empty label file
        else:
            with open(label_path, 'w') as f:
                f.write(f"0 {' '.join(map(str, label))}" + '\n')

    print(f"Saved {n_total} images into '{output_dir}' under train/validation/test folders.")




In [3]:

def grayscale_to_rgb(grayscale_array):
    # Assuming grayscale_array is of shape (n, 28, 28)
    # Weights for standard grayscale to RGB conversion (luminosity method)
    #weights = np.array([0.2989, 0.5870, 0.1140])
    
    # Replicate the grayscale to each channel with the weighted sum
    rgb_array = np.stack([grayscale_array] * 3, axis=-1)
    #rgb_array = rgb_array * weights  # Apply the weights
    return rgb_array

In [3]:
#use the noisy plus the regular mnist data as the set of images - labels should be the boxes from the masks for first 59995 + 10010 and empty for last 70000
corrupted_imgs = np.empty((0, 28, 28))
corrupted_labels = np.empty((0, 4))
for i in range(0, 56000, 4615): 
    #take from i to i+4614 for the training set and set its label to 1 (for corrupted = true) 
    file_num = int((i/4615)*5 + 10) # get the number of the thing we should be on 
    current_np = np.load("mnist_noisy/mnist_" + str(file_num) + ".npy")
    current_labels = np.load("yolo_boxes/boxes_" + str(file_num) + ".npy")
    corrupted_imgs = np.concatenate((corrupted_imgs, current_np[i:i+4615]), axis = 0)
    corrupted_imgs = np.concatenate((corrupted_imgs, current_np[69230:]), axis = 0)
    corrupted_labels = np.concatenate((corrupted_labels, current_labels[i:i+4615]), axis = 0)
    corrupted_labels = np.concatenate((corrupted_labels, current_labels[69230:]), axis = 0)
color_img = grayscale_to_rgb(corrupted_imgs)
save_images_labels(color_img, corrupted_labels, "yolo_data/images3", "yolo_data/labels")


In [4]:
#now do this for the basic mnist with empty text files for the labels - start the file numbering at 70005
mnist_images = np.load("mnist_files/mnist_img.npy")
mnist_labels = np.zeros((mnist_images.shape[0], 4))
print(mnist_labels.shape)
mnist_color = grayscale_to_rgb(mnist_images)

save_images_labels(mnist_color, mnist_labels, "yolo_data/images", "yolo_data/labels", 70005)



(70000, 4)


In [7]:
total_test = np.concatenate((corrupted_labels, mnist_labels), axis=0)

In [10]:
print(total_test.shape)
first_test = total_test[63004:70005]
second_test = total_test[133005:140005]
total_test = np.concatenate((first_test, second_test), axis=0)
print(total_test.shape)



(140005, 4)
(14001, 4)


In [11]:
np.save("mnist_noisy/all_labels", total_test) 

In [16]:
#generate new test files - all with occlusions none without -- for non-shuffled test cases 
corrupted_imgs = np.empty((0, 28, 28))
corrupted_labels = np.empty((0, 4))
for i in range(0, 56000, 4615): 
    #take from i to i+4614 for the training set and set its label to 1 (for corrupted = true) 
    file_num = int((i/4615)*5 + 10) # get the number of the thing we should be on 
    current_np = np.load("mnist_noisy/mnist_" + str(file_num) + ".npy")
    current_labels = np.load("yolo_boxes/boxes_" + str(file_num) + ".npy")
    corrupted_imgs = np.concatenate((corrupted_imgs, current_np[i+4615:i+9230]), axis = 0)
    corrupted_labels = np.concatenate((corrupted_labels, current_labels[i+4615:i+9230]), axis = 0)
color_img = grayscale_to_rgb(corrupted_imgs)
save_images_labels(color_img, corrupted_labels, "yolo_data/images2", "yolo_data/labels2", explicit_split = "test" )


KeyboardInterrupt: 

In [20]:
#new training data in case it is needed 
corrupted_imgs = np.empty((0, 28, 28))
corrupted_labels = np.empty((0, 4))
for i in range(0, 56000, 4615): 
    #take from i to i+4614 for the training set and set its label to 1 (for corrupted = true) 
    file_num = int((i/4615)*5 + 10) # get the number of the thing we should be on 
    chunk_start = i + 2 * 4615
    chunk_end = i + 3 * 4615
    current_np = np.load("mnist_noisy/mnist_" + str(file_num) + ".npy")
    current_labels = np.load("yolo_boxes/boxes_" + str(file_num) + ".npy")
    corrupted_imgs = np.concatenate((corrupted_imgs, current_np[chunk_start:chunk_end]), axis = 0)
    corrupted_labels = np.concatenate((corrupted_labels, current_labels[chunk_start:chunk_end]), axis = 0)
color_img = grayscale_to_rgb(corrupted_imgs)
save_images_labels(color_img, corrupted_labels, "yolo_data/images2", "yolo_data/labels2", explicit_split = "train" )

(70000, 28, 28)
(70000, 28, 28)
(70000, 28, 28)
(70000, 28, 28)
(70000, 28, 28)
(70000, 28, 28)
(70000, 28, 28)
(70000, 28, 28)
(70000, 28, 28)
(70000, 28, 28)
(70000, 28, 28)
(70000, 28, 28)
(70000, 28, 28)
Saved 59995 images into 'yolo_data/images2' under train/validation/test folders.


In [21]:
#new training data in case it is needed 
corrupted_imgs = np.empty((0, 28, 28))
corrupted_labels = np.empty((0, 4))
for i in range(0, 56000, 4615): 
    #take from i to i+4614 for the training set and set its label to 1 (for corrupted = true) 
    file_num = int((i/4615)*5 + 10) # get the number of the thing we should be on 
    chunk_start = i + 3 * 4615
    chunk_end = i + 4 * 4615
    current_np = np.load("mnist_noisy/mnist_" + str(file_num) + ".npy")
    current_labels = np.load("yolo_boxes/boxes_" + str(file_num) + ".npy")
    corrupted_imgs = np.concatenate((corrupted_imgs, current_np[chunk_start:chunk_end]), axis = 0)
    corrupted_labels = np.concatenate((corrupted_labels, current_labels[chunk_start:chunk_end]), axis = 0)
color_img = grayscale_to_rgb(corrupted_imgs)
save_images_labels(color_img, corrupted_labels, "yolo_data/images2", "yolo_data/labels2", explicit_split = "validation" )

Saved 56155 images into 'yolo_data/images2' under train/validation/test folders.


In [25]:
# %%bash
# cd yolo_data
# if [ -d "images" ]; then
#     mv "images" "images3"
#     echo "'labels' has been renamed to 'label3'."
# else
#     echo "'labels' folder does not exist."
# fi

# # Rename 'labels2' to 'labels'
# if [ -d "images2" ]; then
#     mv "images2" "images"
#     echo "'labels2' has been renamed to 'labels'."
# else
#     echo "'labels2' folder does not exist."
# fi
# #python train.py --img 64 --batch 16 --epochs 50 --data custom_data.yaml --weights runs/train/exp2/weights/best.pt --cache
# python detect.py --weights runs/train/exp5/weights/best.pt --source ../yolo_data/images/test --save-txt 


'labels' has been renamed to 'label3'.
'labels2' has been renamed to 'labels'.
